<a href="https://colab.research.google.com/github/j03m/lstm-price-predictor/blob/main/stable_baselines_ppo_trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/DLR-RM/stable-baselines3@feat/gymnasium-support
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib@feat/gymnasium-support
!pip install yfinance
!pip install gymnasium


In [4]:
!pip install sb3-contrib

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys

sys.path.insert(0,'/content/drive/My Drive/ml-trde-notebooks')
data_path = '/content/drive/My Drive/ml-trde-notebooks/data'
model_path = "/content/drive/My Drive/ml-trde-notebooks/models"

Mounted at /content/drive


In [6]:
import pandas as pd
import numpy as np
import math

from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import norm

import multiprocessing

from stable_baselines3 import PPO
from sb3_contrib import RecurrentPPO
from stable_baselines3.ppo.policies import MlpPolicy

from stable_baselines3.common.evaluation import evaluate_policy

cores = multiprocessing.cpu_count()
print("I has cores: ", cores)





I has cores:  8


In [7]:
# Executes this notebook in our space, making all of its functions/globals available
%run -i '/content/drive/My Drive/ml-trde-notebooks/backtest-common.ipynb'

Modified
1679922395
1679922395.0
1679922395.0


In [8]:
logging_level = 0
def error(*args):
    if logging_level >= 0:
        print(*args)

def info(*args):
    if logging_level >=1:
        print(*args)

def verbose(*args):
    if logging_level >=2:
        print(*args)

def debug(*args):
    if logging_level >=3:
        print(*args)

In [15]:
def download_stocks(total):
  # We don't want to train against crypto
  return get_all_stock_timerseries_for_csv("training_tickers3.csv", 3500, total)

LENGTH_OF_STOCK_TRAINGING_DATA = 145 #I might need to fix this, but the model is tied to the number of symbols we trained on
def download_crypto():
  return get_all_product_timeseries(-1, 180, LENGTH_OF_STOCK_TRAINGING_DATA)

In [16]:
from_disk = False
use_crypto = False

if from_disk:
  product_data = load_dict(use_crypto)
  first_key = list(product_data.keys())[0]
  length = len(product_data[first_key])
elif use_crypto:
  product_data = download_crypto()
  save_dict(product_data, True)
else:
  product_data = download_stocks(2)
  save_dict(product_data)

min_len = min(len(df) for df in product_data.values())
print("symbols:", len(product_data.keys()))
print("min length:", min_len)
for name, df in product_data.items():
    product_data[name] = df.head(min_len)

product_data = dict(sorted(product_data.items(), reverse=True))

loading ticker:  XTSLA  count: 0 of: 164
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- XTSLA: No timezone found, symbol may be delisted
loading ticker:  XEL  count: 1 of: 164
[*********************100%***********************]  1 of 1 completed
loading ticker:  WU  count: 2 of: 164
[*********************100%***********************]  1 of 1 completed
loading ticker:  WTW  count: 3 of: 164
symbols: 2
min length: 3500


In [12]:
import numpy as np
import gymnasium as gym
from gymnasium import logger, spaces

class TraderEnv(gym.Env):
    
    def __init__(self, product, df):
        super(TraderEnv, self).__init__()

        # Define the bounds for each column
        close_min, close_max = 0, np.inf  
        volume_min, volume_max = 0, np.inf  
        trend_min, trend_max = 0, np.inf  
        percentile_min, percentile_max = 0, 1

        # Define the observation space
        self.observation_space = spaces.Box(low=np.array([close_min, volume_min, trend_min, percentile_min]),
                                      high=np.array([close_max, volume_max, trend_max, percentile_max]),
                                      dtype=np.float32)

        # We have 3 actions, hold (0), long (1), short (2)
        self.action_space = spaces.Discrete(3)
        
        # Initialize environment state
        df = self.expand(df.copy())

        self.orig_timeseries = df
        self.timeseries = self.scale(df[["Date", "Close", "weighted-volume", "trend", "prob_above_trend"]])
        
        self._reset_vars()
        self.calculate_benchmark_metrics()


        # Don't add this here. Process the tickers into data files - you already have some.
        #self.lstm_internal = keras.models.load_model(model_path + "/lstm_cv-4.h15")
        #predictions = self.predict_trade(self.lstm_internal, self.timeseries[["Close", "weighted-volume"]])
        #self.timeseries["predicted-price"] = predictions
        
        self.product = product
        self.final = len(df)
    
    
    def calculate_benchmark_metrics(self):
      df = self.orig_timeseries
      row = df.iloc[0,:]
      price = self.get_price_with_slippage(row["Close"])
      self.benchmark_position_shares = math.floor(self.initial_cash/price)
     
    def predict_trade(self, model, X):
      predicted = model.predict(X, verbose=0).flatten()
     
      return predicted

    def expand(self, df):
      # Perform seasonal decomposition
      result = seasonal_decompose(df['Close'], model='additive', period=90, extrapolate_trend='freq')

      # Add trend back to original time series
      df["trend"] = result.trend

      # Compute the residuals by subtracting the trend from the original time series
      residuals = result.resid

      # Fit a Gaussian distribution to the residuals
      mu, std = norm.fit(residuals)

      # Compute the probability of a value being above or below the trend line
      # for each point in the time series
      z_scores = residuals / std
      df["prob_above_trend"] = 1 - norm.cdf(z_scores)
      df["weighted-volume"] = df["Close"] * df["Volume"]
      return df

    def scale(self, timeseries):
      df = timeseries.reset_index()  # Reset the index of the DataFrame
      dates = df['Date']
      data_to_scale = df.drop(['Date', 'index'], axis=1)
      self.scaler = MinMaxScaler()
      scaled_data = self.scaler.fit_transform(data_to_scale)
      scaled_df = pd.concat([dates, pd.DataFrame(scaled_data, columns=data_to_scale.columns)], axis=1)
      return scaled_df.set_index('Date')  # Set the index back to 'Date'

    def make_ledger_row(self):
        ledger = pd.DataFrame()
        ledger["Date"] = []
        ledger["Side"] = []
        ledger["Action"]  = []
        ledger["Price"] = []
        ledger["Fee"] = []
        return ledger

    # def action_spec(self):
    #     return self._action_spec

    # def observation_spec(self):
    #     return self._observation_spec

    def _reset_vars(self):
        self._state = None
        self._episode_ended = False        
        self.ledger = self.make_ledger_row()
        self.slippage = .01
        self.fee = .0025
        self.current_index = 0
        self.cash = 5000
        self.initial_cash = self.cash
        self.position_shares = 0
        self.position_value = 0
        self.shares_owed = 0
        self.in_position = False
        self.in_long = False
        self.in_short = False
        self._state = self._get_initial_state()

    def reset(self):
        # Reset the environment and return the initial time step
        self._reset_vars()
        return self._get_next_state(), {}

    def step(self, action):
        action = int(action)
        info("_step:", action)
        
        if self._episode_ended:
            # The last action ended the episode. Ignore the current action and start a new episode.
            return self.reset()

        # Apply the action and update the environment state
        self._apply_action(action)
        self._state = self._get_next_state()

        if self._is_episode_ended():
            self._episode_ended = True
            reward = self._get_final_reward()
            return self._get_next_state(), reward, False, True, {}
        else:
            reward = self._get_reward()
            return self._get_next_state(), reward, False, False, {}

    def _get_initial_state(self):
        # Return the initial state of the environment
        self.current_index = 0
        return self.env_block()

    def total_value(self):
      return self.position_value + self.cash

    def should_stop(self):
      # if cash is negative
      if (self.cash < 0):
        info("Bankrupt.")
        return True
      return False


    def _apply_action(self, action):
        # Advance the environment by one time step and return the observation, reward, and done flag
        self.current_index += 1
        info("step:", "index:", self.current_index, " of: ", self.final-1, "action: ", int(action))
        self.update_position_value()
        if (self.current_index >= self.final - 1 or  self.should_stop()):
          error("********MARKING DONE", "index:", self.current_index, " of: ", self.final-1, " cash: ", self.cash, " value: ", self.position_value)
          if (self.position_shares != 0):
            info("done so closing position")
            self.close_position()
          self._episode_ended = True
        else:
          self._episode_ended = False

        # AI says hold
        if (action == 0):
          info("holding action = 0.")
        # AI says long but we're already in a position
        elif (action == 1 and self.in_long):
          info("holding long.")
        # AI says long, we're not in a position, so buy
        elif (action == 1 and not self.in_position):
          info("opening long.")
          self.open_position()
        #AI says long, but we're short. Close the short, open a long.
        elif (action == 1 and self.in_short):
          info("closing short to open long.")
          self.close_short() 
          self.open_position()
        #AI says short, but we're already short  
        elif (action == 2 and self.in_short):
          info("holding short.")
        #AI says short and we're not in a position so exit 
        elif (action == 2 and not self.in_position):
          info("opening short.")
          self.open_short()
        #AI says short but we're long, close it
        elif (action == 2 and self.in_long):
          info("closing long to open short")
          self.close_position()
          info("opening short.")
          self.open_short()
        else: #assume hold
          info("unknown state! holding:", action, self.in_position, self.in_long, self.in_short)      
        self.update_position_value()
    
    def _get_next_state(self):
        # Calculate and return the next state based on the current state and action taken
        return self.env_block()

    def get_price_with_slippage(self, price):
        return price + (price * self.slippage)

    def open_position(self):
      self.in_position = True
      self.in_long = True
      df = self.orig_timeseries
      row = df.iloc[self.current_index,:]
      price = self.get_price_with_slippage(row["Close"])
      self.position_shares = math.floor(self.cash/price)
      self.cash = 0      
      ledger_row = self.make_ledger_row()
      ledger_row["Side"] = ["long"]
      ledger_row["Action"] = ["enter"]
      ledger_row["Price"] = [price]
      ledger_row["Fee"] = [self.fee]
      self.ledger = pd.concat([self.ledger, ledger_row])

    def open_short(self):
      self.in_position = True
      self.in_short = True
      df = self.orig_timeseries
      row = df.iloc[self.current_index,:]
      price = self.get_price_with_slippage(row["Close"])
      max_short_pos = math.floor(self.cash/price)
      self.shares_owed = max_short_pos
      self.cash = self.cash + (self.shares_owed * price)    
      info("Added cash on short: ", self.shares_owed * price, " total: ", self.cash, " took share debt:", self.shares_owed)  
      ledger_row = self.make_ledger_row()
      ledger_row["Side"] = ["short"]
      ledger_row["Action"] = ["enter"]
      ledger_row["Price"] = [price]
      ledger_row["Fee"] = [self.fee]
      self.ledger = pd.concat([self.ledger, ledger_row])

    def close_position(self):
      self.in_position = False
      self.in_long = False
      df = self.orig_timeseries
      row = df.iloc[self.current_index,:]
      price = self.get_price_with_slippage(row["Close"])
      value = price * self.position_shares
      self.position_shares = 0
      self.cash = self.cash + value
      ledger_row = self.make_ledger_row()
      ledger_row["Side"] = ["long"]
      ledger_row["Action"] = ["exit"]
      ledger_row["Price"] = [price]
      ledger_row["Fee"] = [self.cash * self.fee]
      self.ledger = pd.concat([self.ledger, ledger_row])

    def close_short(self):
      self.in_position = False
      self.in_short = False
      df = self.orig_timeseries
      row = df.iloc[self.current_index,:]
      price = self.get_price_with_slippage(row["Close"])
      value = price * self.shares_owed
      self.shares_owed = 0
      self.cash = self.cash - value
      ledger_row = self.make_ledger_row()
      ledger_row["Side"] = ["short"]
      ledger_row["Action"] = ["exit"]
      ledger_row["Price"] = [price]
      ledger_row["Fee"] = [self.cash * self.fee]
      self.ledger = pd.concat([self.ledger, ledger_row])


    def _is_episode_ended(self):
        return self._episode_ended
        
    def update_position_value(self):
      df = self.orig_timeseries
      row = df.iloc[self.current_index,:]
      self.position_value = (row["Close"] * self.position_shares) - (row["Close"] * self.shares_owed)
      self.benchmark_value = row["Close"] * self.benchmark_position_shares
      info("Calculating position value:",
            "current price:", row["Close"],
            "long shares:", self.position_shares,
            "short shares:", self.shares_owed,
            "long value:", row["Close"] * self.position_shares,
            "short debt:", row["Close"] * self.shares_owed,
            "position net value:", self.position_value)

    def _get_reward(self):
        self.update_position_value()
        current_portfolio_value = self.total_value()
        #compare to what we started with:
        #percentage_change = (current_portfolio_value - self.initial_cash) / self.initial_cash
        
        #compare to a bench mark
        percentage_change = ((current_portfolio_value - self.benchmark_value) / self.benchmark_value) * 100
        info("reward states: ",
              "position value: ", self.position_value,
              "shares: ", self.position_shares,
              "cash: ", self.cash,
              "benchmark value:", self.benchmark_value,
              "percentage_change: ", percentage_change)
        
        return percentage_change

    def _get_final_reward(self):
        # Calculate and return the final reward when the episode ends
        return self._get_reward()

    def env_block(self):
      start_index = self.current_index
      end_index = self.current_index
      df = self.timeseries.reset_index().drop(['Date'], axis=1)
      block = df.iloc[start_index].to_numpy()
      return block


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import os
env_count = 0
num_parallel_envs = 4
learning_rate = 3e-4
num_epochs = 5 #25
discount_factor = 0.99
gradient_clipping = 0.5
entropy_regularization = 1e-2
value_pred_loss_coef = 0.5
use_gae = True
use_td_lambda_return = True
actor_loss_weight = 1.0
value_loss_weight = 0.5
log_interval = 5000
eval_interval = 100
train_model = True

def create_env():
    global env_count
    products = list(product_data.keys())
    product = products[env_count]
    env = TraderEnv(product, product_data[product])
    env_count+=1
    return env

def timestr():
  import datetime
  current_time = datetime.datetime.now()
  formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S") 
  return formatted_time

# Create the environment
all_envs = [create_env() for _ in range(len(product_data))]
sorted_envs = sorted(all_envs, key=lambda env: env.product)

if train_model:
  # Train the agent for 10000 steps
  #env = gym.make('CartPole-v1')
  env = TraderEnv("XEL", product_data["XEL"])
  #model = PPO(MlpPolicy, env, verbose=0)
  model = RecurrentPPO("MlpLstmPolicy", env, verbose=1)
  model.learn(total_timesteps=10000)
  model.save(os.path.join(model_path, "baseline-recurrent-ppo"))
  
else:
  env = TraderEnv("WU", product_data["WU"])
  model = RecurrentPPO.load(os.path.join(model_path, "baseline-recurrent-ppo"))
  mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
  print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
